In [ ]:
# IPython magig  tools
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np

from neo.core import SpikeTrain
from quantities import ms, s
from elephant.statistics import mean_firing_rate
from elephant.statistics import time_histogram

import os 
import pandas as pd
import seaborn as sns
sns.set_context('talk')

import sys
sys.path.append("G:/My Drive/WORKING_MEMORY/EXPERIMENTS/ELECTROPHYSIOLOGY/ANALYSIS/src/functions/")
import ephys_functions as ephys

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings. filterwarnings('ignore', category=UserWarning)
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None  # default='warn'

**Check the synch data from the paper**

In [ ]:
df = pd.read_csv("G:/My Drive/WORKING_MEMORY/PAPER/ANALYSIS_Figures/instantaneous firing rate_alltrial_5ms.csv", index_col=0)
df_final = pd.read_csv("G:/My Drive/WORKING_MEMORY/PAPER/WM_manuscript_FIGURES/Fig. 7. Synch/synch_data_trials_2beforeSti.csv", index_col=0)

In [ ]:
animal = "E22_2022-01-13_16-34-24.csv"
for animal in df_final.animal.unique():
    # file_name = 'synch_data_trials_2beforeSti'
    # df_final = pd.read_csv(file_name+'.csv', index_col=0)
    threshold = 0.5

    df_session = df_final.loc[df_final.animal==animal]

    fig, ax = plt.subplots(2,1 , figsize=(10,6))
    panel = ax[0]
    panel1 = ax[1]


    sns.lineplot(x="trial", y="synch_window",data=df_session, color='black',ci=68,ax=panel)      
    sns.lineplot(x="trial", y="synch",data=df_session, color='grey',ci=68,ax=panel1)      
    for plot in [panel, panel1]:
        plot.set_ylabel('Synch')
        plot.set_ylim(0.9,max(df_session.synch_window)+0.3)
        plot.fill_between(df_session['trial'],0.9 , 2.5, where=df_session['WM_roll'] <= threshold,
                    facecolor='indigo', alpha=0.3)
        plot.fill_between(df_session['trial'], 0.9, 2.5,  where=df_session['WM_roll'] >= threshold,
                        facecolor='darkgreen', alpha=0.3)
    panel.set_xlabel('Trials')
    panel.set_title('Mouse E22 13-01')
    plt.show()

**Re-do to obtain activity per bin for all neurons**

In [ ]:
#Path to sessions
path = r'E:\Ephys\summary_complete'

In [ ]:
# List of variables we want to recover
bin_size = 0.02
concat_df = pd.DataFrame()
align = 'Stimulus_ON'
for filename in os.listdir(path):
    print(filename)   

    if filename[-3:] != 'pdf':
        df = pd.read_csv(path + "/"+ filename, sep=',',index_col=0)
    else:
        continue
    
    # Add time before stimulus from the previous trial
    df = ephys.add_time_before_stimulus(df, 4)
    
    for align, start, stop, delay_epoch in zip([align],[-2],
                                               [0],[False]):

        # Sort per trial
        df = df.sort_values('trial')
        # Get unique neurons in the whole session
        unique_neurons = df.cluster_id.unique()
        
        #Create new aligment to the end of the session
        df['a_'+align] = df.fixed_times-df[align]

        # Define bin edges (e.g., 20 ms bins up to max time)
        bins = np.arange(-2, 0 + bin_size, bin_size)
        bin_labels = bins[:-1]  # Labels for bins (start times)

        # Assign each spike to a time bin
        df['a_'+align + '_bin'] = pd.cut(df['a_' + align], bins=bins, labels=bin_labels, include_lowest=True)
        
        # Skip sessions with less than 30 neurons
        total_neurons_session = len(df.cluster_id.unique())
        if total_neurons_session < 30:
            continue
            
        for T in df.trial.unique(): 
            # Select the trial that we want to look at this time
            dft = df.loc[df.trial ==T]
            
            # Create a DataFrame with all possible combinations of cluster_id and a_END_bin
            all_combinations = pd.MultiIndex.from_product([unique_neurons, bin_labels], names=['cluster_id', 'a_' + align + '_bin']).to_frame(index=False)
        
            # Merge with the actual data to fill missing combinations with NaNs
            binned_spikes = pd.merge(all_combinations, dft.groupby(['cluster_id', 'a_' + align + '_bin']).size().reset_index(name='spike_count'), on=['cluster_id', 'a_' + align + '_bin'], how='left')

            # Fill NaNs in spike_count with 0
            binned_spikes['spike_count'].fillna(0, inplace=True)

            # Convert 'time_bin' back to numeric
            binned_spikes['a_' + align + '_bin'] = binned_spikes['a_' + align + '_bin'].astype(float)
            binned_spikes['trial'] = T
            binned_spikes['session'] = filename
            binned_spikes['WM_roll'] = dft.WM_roll.unique()[0]
            binned_spikes['hit'] = dft.hit.unique()[0]
            concat_df = pd.concat([concat_df, binned_spikes], axis=0)

In [ ]:
concat_df['WM'] = np.where(concat_df['WM_roll'] > 0.5, 1, 0)
concat_df.drop(columns=['WM_roll'], inplace=True)

In [ ]:
df = concat_df.drop_duplicates(subset=['session', 'cluster_id', 'trial', 'WM'])
grp1 = df.groupby(['session', 'WM', 'cluster_id']).trial.nunique().reset_index()
grp2 = grp1.groupby(['session', 'WM']).agg(mean_trial = ('trial', 'mean'), std_trial = ('trial', 'std')).reset_index()
grp2.std_trial.unique()

In [ ]:
concat_df.to_csv(r'C:\Users\tiffany.ona\Documents\working_memory\data\3_11_Review_Prepare the firing rate for rank analysis\20ms_rank_df_fixed_V2.csv')

## Chekc why the two datasets don't have the same number of trials

In [ ]:
df_final = pd.read_csv("G:/My Drive/WORKING_MEMORY/PAPER/WM_manuscript_FIGURES/Fig. 7. Synch/synch_data_trials_2beforeSti.csv", index_col=0)
df_final['state'] = np.where(df_final['WM_roll']>0.5, 0, 1)
concat_df = pd.read_csv(r'C:\Users\tiffany.ona\Documents\working_memory\data\3_11_Review_Prepare the firing rate for rank analysis\20ms_rank_df_fixed_V2.csv', index_col=0)

## Reobtain the synch data

In [ ]:
def compute_window_centered(data, runningwindow,option, trials='trials'):
    """
    Computes a rolling average with a length of runningwindow samples.
    """
    performance = []
    start_on=False
    for i in range(len(data)):
        if data[trials].iloc[i] < int(runningwindow/2):
            # Store the first index of that session for the first initial trials
            if start_on == False:
                start=i
                start_on=True
            performance.append(round(np.mean(data[option].iloc[start:i + int(runningwindow/2)]), 3))
        elif i < (len(data)-runningwindow):
            if data[trials].iloc[i] > data[trials].iloc[i+runningwindow]:
                # Store the last values for the end of the session
                if end == True:
                    end_value = i+runningwindow-1
                    end = False
                performance.append(round(np.mean(data[option].iloc[i:end_value]), 3))
                
            else: # Rest of the session
                start_on=False
                end = True
                performance.append(round(np.mean(data[option].iloc[i - int(runningwindow/2):i+int(runningwindow/2)]), 3))
            
        else:
            performance.append(round(np.mean(data[option].iloc[i:len(data)]), 3))

    return performance

In [ ]:
# List of variables we want to recover
mean_firing= []
cv=[]
synch = []
repeat_list=[]
hit_list=[]
wm_list=[]
animal_list = []
T_max = []
trial = []
synch_mean_window=[]
aligment_window=[]

surrogates=100

In [ ]:
for filename in os.listdir(path):
    print(filename)   

    if filename[-3:] != 'pdf':
        df = pd.read_csv(path + "/"+ filename, sep=',',index_col=0)
    else:
        continue
        
    print(filename)   
    
    # Add time before stimulus from the previous trial
    df = ephys.add_time_before_stimulus(df, 4)
    
#################################################### --------------------------------------------------------------------
    # for align, start, stop, delay_epoch in zip(['Stimulus_ON','Delay_OFF','Delay_OFF','Lick_ON'],[-2,-2,0,-0.25],
    #                                            [0,0,2,1.5],[False, True, False, False]):
    for align, start, stop, delay_epoch in zip(['Stimulus_ON'],[-2],
                                               [0],[False]):

        #Create new aligment to the end of the session
        df['a_'+align] = df.fixed_times-df[align]

        # Sort per trial
        df = df.sort_values('trial')

        #Filter for the last 2 seconds of the ITI
        # df = df.loc[(df['a_'+align]>start)&(df['a_'+align]<stop)]

        # total_neurons_session = len(df.loc[(df.trial_start<df.trial)&(df.trial_end>df.trial)].cluster_id.unique())
        total_neurons_session = len(df.cluster_id.unique())
        if total_neurons_session < 30:
            continue
            
        print('There are ' + str(total_neurons_session) + ' neurons')

        for T in df.trial.unique(): 
            # Select the trial that we want to look at this time
            dft = df.loc[df.trial ==T]
                
            dft = dft.loc[(dft['a_'+align]>start)&(dft['a_'+align]<stop)]

            # Recover amount of neurons that were being registered at that trial interval
    #         n_neurons = len(df.loc[(df.trial_start<T)&(df.trial_end>T)].cluster_id.unique())
            n_neurons = len(df.cluster_id.unique())

            hit_list.append(dft.hit.unique()[0])
            repeat_list.append(dft.repeat_choice.unique()[0])
            wm_list.append(dft.WM_roll.unique()[0])

            if delay_epoch:
                # if dft.delay.unique()[0] == 0.1 or dft.delay.unique()[0] == 0.2:
                if dft.delay.unique()[0] != 1 and dft.delay.unique()[0] == 3 and dft.delay.unique()[0] == 3:
                    mean_firing.append(np.nan)
                    synch.append(np.nan)
                    trial.append(T)
                    animal_list.append(filename)
                    aligment_window.append(align+'_'+str(start)+'_'+str(stop))
                    T_max.append(max(df.trial.unique()))
                    continue
                    
            times_spikes = dft['a_'+align].values
            times_spikes = times_spikes*1000*ms #transform to ms

            ############################################################ Set the strat and end time of the train
            stop_time =  stop*1000*ms ## End of the trial in ms
            start_time = start*1000*ms ## Start of the trial in ms     

            ############################################################ Spiketrain
            spiketrain = SpikeTrain(times_spikes, units=ms, t_stop=stop_time, t_start=start_time) 

            ############################################################ 
            histogram_rate = time_histogram([spiketrain], 20*ms, output='rate')
            times_ = histogram_rate.times.rescale(s)
            firing_real = histogram_rate.rescale(histogram_rate.dimensionality).magnitude.flatten()

            mean_firing.append(float(mean_firing_rate(spiketrain)/n_neurons*1000))
    #         print("The mean firing rate of spiketrain is", mean_firing_rate(spiketrain)/n_neurons)

    #         print("The std of the firing rate of spiketrain is", np.std(firing))
            real_std=np.std(firing_real) # Store the real std value

            # t1_start = process_time() 
            list_std = []
            for i in range(surrogates):
                # Create a random shuffle for the same amount of spikes in that interval
                random_float_list = np.random.uniform(start, stop, len(times_spikes))
                # t1_stop = process_time() 
                # print(str(T) + ' ' + str(t1_stop-t1_start))
                
                surrogate_spikes = np.array(random_float_list)*1000*ms #transform to ms
                spiketrain = SpikeTrain(surrogate_spikes, units=ms, t_stop=stop_time, t_start=start_time) 

                histogram_rate = time_histogram([spiketrain], 20*ms, output='rate')
                times_ = histogram_rate.times.rescale(s)
                firing = histogram_rate.rescale(histogram_rate.dimensionality).magnitude.flatten()

                list_std.append(np.std(firing))

            synch.append(real_std/np.mean(list_std))
            
            trial.append(T)
            animal_list.append(filename)
            aligment_window.append(align+'_'+str(start)+'_'+str(stop))
            T_max.append(max(df.trial.unique()))


In [ ]:
fig, axes = plt.subplots(6,2 , figsize=(12,20))
for session, ax in zip(df_final.animal.unique(), axes.flatten()):
    sns.lineplot(x="trials", y="FR",data=df_final.loc[df_final.animal == session],ax=ax)

In [ ]:
# Create new dataframe with all the recovered lists. 
df_final = pd.DataFrame(list(zip(mean_firing, synch, hit_list, wm_list, repeat_list, T_max,trial,animal_list, aligment_window)), 
       columns =['FR', 'synch','hit','WM_roll','repeat','total_trials','trials','animal', 'aligment'])

df_final.dropna(inplace=True)

df_final['accuracy'] = compute_window_centered(df_final,10,'hit')
df_final['repeat_bias'] = compute_window_centered(df_final,10,'repeat')
df_final['synch_window']= compute_window_centered(df_final,5,'synch')